In [37]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [38]:
%pip install -q decorator==5.0.9
!pip install -q ipywidgets

import re
import imp
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from collections import Counter

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Note: you may need to restart the kernel to use updated packages.


In [39]:
%pip install -q cltk==1.0.22

Note: you may need to restart the kernel to use updated packages.


In [40]:
# importing local modules

import sys
sys.path.append('../src/')

import text_cleanup.text_cleanup as thesisCleanUp
import preprocessing.text_preprocessing as thesisTextPreprocessing
import data.reader as thesisDataReader
import utils.utils as thesisUtils
import features.tf_idf.n_gram as thesisTfIdfNgramFeatures
import features.count_vectorizer.n_gram as thesisCountVectorizerNgramFeatures
import similarities.cosine as thesisCosineSimilarities
import vocabulary.vocabulary as thesisVocabulary
import features.lexical as thesisLexicalFeatures
import similarities.cosine as thesisCosineSimilarity
import text_cleanup.text_cleanup as thesisTextCleanUp
import p_aligment.p_aligment as thesisPAligment

imp.reload(thesisLexicalFeatures)
imp.reload(thesisCosineSimilarity)
imp.reload(thesisCleanUp)
imp.reload(thesisTextPreprocessing)
imp.reload(thesisDataReader)
imp.reload(thesisUtils)
imp.reload(thesisTfIdfNgramFeatures)
imp.reload(thesisVocabulary)
imp.reload(thesisCosineSimilarities)
imp.reload(thesisTextCleanUp)
imp.reload(thesisCountVectorizerNgramFeatures)
imp.reload(thesisPAligment)

<module 'p_aligment.p_aligment' from '../src/p_aligment/p_aligment.py'>

# shared vocabulary suffix research

In [41]:
london_breslau_shared_words = pd.read_csv('../computed_data/vocabulary/shared_2_versions/london_breslau.csv').drop(['Unnamed: 0'], axis=1)
london_zwickau_shared_words = pd.read_csv('../computed_data/vocabulary/shared_2_versions/london_zwickau.csv').drop(['Unnamed: 0'], axis=1)
zwickau_breslau_shared_words = pd.read_csv('../computed_data/vocabulary/shared_2_versions/zwickau_breslau.csv').drop(['Unnamed: 0'], axis=1)

In [42]:
london_zwickau_shared_words

,shared_word,version a name,version a count,version b name,version b count
0,iacere,london,19,zwickau,14
1,longum,london,15,zwickau,10
2,oppidum,london,15,zwickau,1
3,ueni,london,15,zwickau,10
4,campo,london,13,zwickau,7
...,...,...,...,...,...
2794,terrere,london,1,zwickau,1
2795,margat,london,1,zwickau,2
2796,ualaniam,london,1,zwickau,1
2797,doram,london,1,zwickau,1


In [62]:
def count_suffix(corpus, suffix):
    result = []
    
    for i, p in enumerate(corpus):
        matches = 0
        
        for w in p.split():
            if w.endswith(suffix): matches += 1
                
        if matches > 0:
            result.append({'suffix': suffix, 'p_index': i, 'matches': matches})
    
    return result

In [63]:
SUFFIX_LEN = 4
suffixes = set()
for index, row in london_breslau_shared_words.iterrows():
    word = row['shared_word']
    if len(word) <= SUFFIX_LEN: continue
        
    suffix = word[-SUFFIX_LEN:]
    suffixes.add(suffix)
    print(f'{word} - {suffix}')

len(suffixes)

pilistiim - tiim
deinde - inde
quatuor - tuor
acconensi - ensi
magedo - gedo
musiuo - siuo
esecias - cias
suficient - ient
descensum - nsum
posita - sita
subter - bter
rapaim - paim
cerubin - ubin
occidenti - enti
austri - stri
impetu - petu
descendentes - ntes
celebraui - raui
altissimo - simo
grecis - ecis
monti - onti
duorum - orum
dispositus - itus
hermoniim - niim
maceronta - onta
neapolis - olis
pesec - esec
neccessarie - arie
sennacerib - erib
duplici - lici
duabus - abus
betsam - tsam
ackaron - aron
iudicatum - atum
emitentes - ntes
deserentem - ntem
transuolantem - ntem
consecrauit - auit
surianis - anis
querebam - ebam
interrogans - gans
legentibus - ibus
lineas - neas
metropolim - olim
regalis - alis
plurima - rima
directam - ctam
penitia - itia
arcis - rcis
lamperti - erti
impelunt - lunt
contiguam - guam
portauit - auit
demetrium - rium
australem - alem
crederentur - ntur
ceperunt - runt
paneas - neas
masrepa - repa
amorreos - reos
manasses - sses
iahel - ahel
fugiciuorum 

120

In [64]:
#     result = {}

#     resp = count_suffix(thesisDataReader.get_london_by_new_line(), suffix)
#     if resp is not None: 
#         print(resp)
#         print()
results = []
for suffix in suffixes:
    resp_london = count_suffix(thesisDataReader.get_london_by_new_line(), suffix)
    resp_breslau = count_suffix(thesisDataReader.get_breslau_by_new_line(), suffix)
    resp_zwickau = count_suffix(thesisDataReader.get_zwickau_by_new_line(), suffix)
    
#     if resp_london is not None: 
#         print('london result:')
#         print(resp_london)
#         print()
        
#     if resp_breslau is not None: 
#         print('breslau result:')
#         print(resp_breslau)
#         print()
    
#     if resp_zwickau is not None:
#         print('zwickau result:')
#         print(resp_zwickau)
#         print()
        
    results.append([resp_london, resp_breslau, resp_zwickau])
results

[[[{'suffix': 'rmes', 'p_index': 145, 'matches': 1}],
  [{'suffix': 'rmes', 'p_index': 44, 'matches': 1}],
  []],
 [[{'suffix': 'cias', 'p_index': 169, 'matches': 1},
   {'suffix': 'cias', 'p_index': 171, 'matches': 3},
   {'suffix': 'cias', 'p_index': 227, 'matches': 1}],
  [{'suffix': 'cias', 'p_index': 54, 'matches': 1}],
  []],
 [[{'suffix': 'nuum', 'p_index': 176, 'matches': 2},
   {'suffix': 'nuum', 'p_index': 225, 'matches': 1}],
  [{'suffix': 'nuum', 'p_index': 56, 'matches': 1}],
  [{'suffix': 'nuum', 'p_index': 185, 'matches': 1}]],
 [[{'suffix': 'deon', 'p_index': 107, 'matches': 1}],
  [{'suffix': 'deon', 'p_index': 36, 'matches': 1}],
  []],
 [[{'suffix': 'onis', 'p_index': 3, 'matches': 2},
   {'suffix': 'onis', 'p_index': 4, 'matches': 1},
   {'suffix': 'onis', 'p_index': 5, 'matches': 1},
   {'suffix': 'onis', 'p_index': 18, 'matches': 1},
   {'suffix': 'onis', 'p_index': 20, 'matches': 1},
   {'suffix': 'onis', 'p_index': 21, 'matches': 1},
   {'suffix': 'onis', 'p_ind

In [76]:
fig, axes = plt.subplots(len(results), 1, figsize=(35,700))

for i, ax in enumerate(axes.ravel()):
    london_result = results[i][0]
    breslau_result = results[i][1]
    zwickau_result = results[i][2]
    
    suffix = london_result[0]['suffix']
    ax.set_title(f'suffix - {suffix}')
#     print(suffix)
#     print(london_result)
    
    if len(london_result) > 0:
        plot_values = []
        for i in london_result:
#             print(i)
            plot_values.insert(i['p_index'], i['matches'])
#             plot_values[i] = r
#         print(plot_values)
        ax.set_xlim([0, 350])
        ax.plot(plot_values, label='london')

In [28]:
result

{50: {'suffix': 'blin', 'p_index': 50, 'matches': 1},
 316: {'suffix': 'blin', 'p_index': 316, 'matches': 1}}